In [9]:
import tensorflow as tf
import keras


CLASS_NAMES = [
    "ADI",
    "BACK",
    "DEB",
    "LYM",
    "MUC",
    "MUS",
    "NORM",
    "STR",
    "TUM",
]

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "NCT-CRC-HE-small",
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_NAMES,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "NCT-CRC-HE-small",
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_NAMES,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
)


Found 6300 files belonging to 9 classes.
Using 4410 files for training.
Found 6300 files belonging to 9 classes.
Using 1890 files for validation.


In [19]:
import os
os.environ["TF_XLA_FLAGS"] = "-tf_xla_cpu_global_jit"

https://keras.io/api/applications/

In [23]:
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False) 

# freeze the base model
base_model.trainable = False

# create a new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(9, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# fine-tune the model
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# train the model
model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
)

94765736/94765736 [==============================] - 12s 0us/step
Epoch 1/2
138/138 [==============================] - 19s 123ms/step - loss: 0.6221 - accuracy: 0.7957 - val_loss: 0.3695 - val_accuracy: 0.8825
Epoch 2/2
138/138 [==============================] - 15s 108ms/step - loss: 0.2882 - accuracy: 0.9079 - val_loss: 0.2902 - val_accuracy: 0.9053
